In [1]:
import json
import cv2
from shapely.geometry import Polygon
import itertools
import shutil
import os
from detectron2.structures import BoxMode
import matplotlib.pyplot as plt
import numpy as np
import PIL
from PIL import Image

c:\Users\user\Desktop\RCP\ClimbAssistant\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Copy files into train and val folders

In [2]:
# we do a train-test split of 80-20 for this iteration
train_num = 79 # use 80 photos for training (out of 100)
count = 0

train_path = "C:/Users/user/Desktop/boulder photos/iter2/train"
val_path = "C:/Users/user/Desktop/boulder photos/iter2/val"

bk_path = "C:/Users/user/Desktop/boulder photos/labelling/bk"
xr_path = "C:/Users/user/Desktop/boulder photos/labelling/xinrui"

In [3]:
# for bk images
for file in os.listdir(bk_path):
    if file.endswith('.jpeg'):
        file_path = os.path.join(bk_path, file)
        shutil.copy(file_path, train_path)
        count +=1 

print(count)

63


In [4]:
# for xr images, some go to validation set
for file in os.listdir(xr_path):
    if file.endswith('.jpeg'):
        file_path = os.path.join(xr_path, file)
        if count <= train_num:
            shutil.copy(file_path, train_path)
        else:
            shutil.copy(file_path, val_path)
        count +=1 
    
print(count)

100


### Extract json data and transform into required COCO format

In [5]:
# first we get the image size
# since all taken in iPhone, we just need to check 1 image
# im = cv2.imread(train_path + "/boulderingwallstockimages0.jpeg")
# h, w, c = im.shape
# print(h, w)

In [6]:
# since we are using only holds for our training, we set the category ID all to the same number
cat_id = 0

In [7]:
# read data from the bk labels file
with open(os.path.join(bk_path, "labels.json")) as f:
    d = json.load(f)

# read data from the xr labels file
with open(os.path.join(xr_path, "labels.json")) as f:
    d1 = json.load(f)

In [8]:
# open train and mask json
with open(os.path.join(train_path, "annotations.json")) as f:
    train = json.load(f)
    
with open(os.path.join(val_path, "annotations.json")) as f:
    val = json.load(f)

In [9]:
# FOR TRAIN SET

# we start image & mask ID count from 0
train_image_count = 0
train_mask_count = 0

# store the images we filtered into train dataset
train_imgs = []

for key in d:
        # get the file name and read the dimensions of each image
        file_name = d[key]['filename']

        file_path = os.path.join(train_path, file_name)

        im = cv2.imread(file_path)
        h, w, c = im.shape

        # this is the dictionary to append to images list
        dict_1 = {
                "file_name": d[key]['filename'],
                "height": h,
                "width": w,
                "id": train_image_count 
                }

        # add into train set
        train['images'].append(dict_1)
        train_imgs.append(d[key]['filename'])

        result = im.copy()
                
        for segment in d[key]['regions']:
                x = segment['shape_attributes']['all_points_x']
                y = segment['shape_attributes']['all_points_y']
                
                # get the area of the polygon from the points
                pgon = Polygon(zip(x, y))
                area = pgon.area
                
                # combine the x and y coordinates into 1 alternating list
                coords = [i for i in itertools.chain.from_iterable(itertools.zip_longest(x,y)) if i]
                
                # get bbox coordinates
                bbox = [min(x), min(y), max(x) - min(x), max(y) - min(y)] # bbox coordinates must be in the XYWH format!!

                # cv2.rectangle(result, (min(x), min(y)), (max(x), max(y)), (0,0,255), 2)
                
                # this is the dictionary to append to annotations list
                dict_2 = {
                        "segmentation": [ coords ],
                        "area": area,
                        "iscrowd": 0,
                        "image_id": train_image_count,
                        "bbox": bbox,
                        "bbox_mode": 0,
                        "category_id": cat_id,
                        "id": train_mask_count
                        }
                
                # this logic is used to split real image dataset equally into train n val
                train['annotations'].append(dict_2)
                
                train_mask_count += 1
        # plt.imshow(result)
        # plt.show()
        train_image_count += 1

In [10]:
# FOR VALIDATION SET
# for xinrui dataset, we need to split into both train n val sets

# we start image & mask ID count from 0
val_image_count = 0
val_mask_count = 0

# store the images we filtered into val dataset
val_imgs = []

# check if the file in train path to add to train annotations, else add to val one
train_dir_imgs = os.listdir(train_path)

for key in d1:
        # get the file name and read the dimensions of each image
        file_name = d1[key]['filename']

        # if the file has been added to the training directory, we will add it to the train annotations
        # else, it will be in the validation annotations
        if file_name in train_dir_imgs:
                file_path = os.path.join(train_path, file_name)
                im = cv2.imread(file_path)
                h, w, c = im.shape

                # this is the dictionary to append to images list
                dict_1 = {
                        "file_name": d1[key]['filename'],
                        "height": h,
                        "width": w,
                        "id": train_image_count 
                        }

                # add into train or val set
                train['images'].append(dict_1)
                train_imgs.append(d1[key]['filename'])

                        
                for segment in d1[key]['regions']:
                        x = segment['shape_attributes']['all_points_x']
                        y = segment['shape_attributes']['all_points_y']
                        
                        # get the area of the polygon from the points
                        pgon = Polygon(zip(x, y))
                        area = pgon.area
                        
                        # combine the x and y coordinates into 1 alternating list
                        coords = [i for i in itertools.chain.from_iterable(itertools.zip_longest(x,y)) if i]
                        
                        # get bbox coordinates
                        bbox = [min(x), min(y), max(x) - min(x), max(y) - min(y)]
                        
                        # this is the dictionary to append to annotations list
                        dict_2 = {
                                "segmentation": [ coords ],
                                "area": area,
                                "iscrowd": 0,
                                "image_id": train_image_count,
                                "bbox": bbox,
                                "bbox_mode": 0,
                                "category_id": cat_id,
                                "id": train_mask_count
                                }
                        
                        # this logic is used to split real image dataset equally into train n val
                        train['annotations'].append(dict_2)
                        
                        train_mask_count += 1
                        
                train_image_count += 1
        
        else:
                file_path = os.path.join(val_path, file_name)
                im = cv2.imread(file_path)
                h, w, c = im.shape

                # this is the dictionary to append to images list
                dict_1 = {
                        "file_name": d1[key]['filename'],
                        "height": h,
                        "width": w,
                        "id": val_image_count 
                        }

                # add into train or val set
                val['images'].append(dict_1)
                val_imgs.append(d1[key]['filename'])

                for segment in d1[key]['regions']:
                        x = segment['shape_attributes']['all_points_x']
                        y = segment['shape_attributes']['all_points_y']
                        
                        # get the area of the polygon from the points
                        pgon = Polygon(zip(x, y))
                        area = pgon.area
                        
                        # combine the x and y coordinates into 1 alternating list
                        coords = [i for i in itertools.chain.from_iterable(itertools.zip_longest(x,y)) if i]
                        
                        # get bbox coordinates
                        bbox = [min(x), min(y), max(x) - min(x), max(y) - min(y)]
                        
                        # this is the dictionary to append to annotations list
                        dict_2 = {
                                "segmentation": [ coords ],
                                "area": area,
                                "iscrowd": 0,
                                "image_id": val_image_count,
                                "bbox": bbox,
                                "bbox_mode": 0,
                                "category_id": cat_id,
                                "id": val_mask_count
                                }
                        
                        # this logic is used to split real image dataset equally into train n val
                        val['annotations'].append(dict_2)
                        
                        val_mask_count += 1
                        
                val_image_count += 1

In [11]:
category = {
            "supercategory": "hold",
            "id": 0,
            "name": "hold"
        }
train['categories'].append(category)
val['categories'].append(category)

In [12]:
# save new info back to train and mask json
with open(os.path.join(train_path, 'annotations.json'), 'w') as f:
    json.dump(train, f)

In [13]:
# save new info back to val and mask json
with open(os.path.join(val_path, 'annotations.json'), 'w') as f:
    json.dump(val, f)